In [ ]:
from collections import OrderedDict
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.models import HoverTool
import math
from bokeh.io import output_notebook
import urllib
import json,pandas

In [ ]:
url = 'https://raw.githubusercontent.com/datasets/geo-boundaries-world-110m/master/countries.geojson'
testfile = urllib.URLopener()
if os.path.exists('countries.json'):
    print "file already exists"
else:
    testfile.retrieve(url, "countries.json")
with open('countries.json','r') as f:
    geodata = json.load(f)

In [ ]:
!wget --quiet --output-document  /resources/EmssionCountry.csv  https://ibm.box.com/shared/static/wycqlfw7brox2su7vks8b8na9aeovxsb.csv
print "...download complete! See your files under 'Recent Data'."

In [ ]:
emission_df= pandas.read_csv('/resources/EmssionCountry.csv')

In [ ]:

geodata_features = geodata['features']
country_xs = []
country_ys = []
country_names = []
country_emission = []
country_colors = []
colors = ['#CCE5FF','#CCCCFF','#9999FF','#6666FF','#3333FF',
         '#0000FF','#0000CC','#000099','#000066','#0000CC']

for aCountry in geodata_features:
    # Set country name
    cName = aCountry['properties']['name']
    
    
    #Set country emission
    if cName in emission_df['CountryName'].values:
        emission_amt = emission_df['year2011'][emission_df[emission_df.CountryName==cName].index[0]]
        if emission_amt>20:
            count_color="#000000"
        else:
            count_color=colors[int(emission_amt*0.4)]
        
    else:
        country_em=0
        count_color="#FFFFFF"

        
    #Set country boundry
    geometry_type = aCountry['geometry']['type']
    if geometry_type == "MultiPolygon":
        for poly_coords in aCountry['geometry']['coordinates']:
            coords = poly_coords[0]
            country_xs.append(map(lambda x:x[0],coords))
            country_ys.append(map(lambda x:x[1],coords))
            country_names.append(cName)
            country_colors.append(count_color)
            country_emission.append(emission_amt)
    else:
        coords = aCountry['geometry']['coordinates'][0]
        country_xs.append(map(lambda x:x[0],coords))
        country_ys.append(map(lambda x:x[1],coords))
        country_names.append(cName)       
        country_colors.append(count_color)
        country_emission.append(emission_amt)        
        
source = ColumnDataSource(
    data = dict(
        x=country_xs,
        y=country_ys,
        color=country_colors,
        name=country_names,
        emission=country_emission,
    )
)

TOOLS="pan,wheel_zoom,box_zoom,reset,hover,save"
p = figure(title="Emission by Country", tools=TOOLS)
p.patches('x', 'y',
    fill_color='color', fill_alpha=0.7,
    line_color="white", line_width=0.5,
    source=source)
hover = p.select(dict(type=HoverTool))
hover.point_policy = "follow_mouse"
hover.tooltips = OrderedDict([
    ("Name", "@name"),
    ("Emission amount", "@emission"),
])
output_notebook()
show(p)